# Quantum bin packing for vehicle routing

Our optimal bin packing solution for vehicle routing ensures efficient space utilization while enabling sequential unloading based on delivery routes for quick retrieval. It optimizes structural stability by considering the center of mass, package orientation, and load-bearing constraints.

This sample notebook shows you how to execute Quantum bin packing for vehicle routing Algorithm from AWS Marketplace

 **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

### Pre-requisites:

1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that IAM role used has **AmazonSageMakerFullAccess**
3. Some hands-on experience using Amazon SageMaker.
4. To use this algorithm successfully, ensure that:  
    A. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the            AWS account used:  
&nbsp;&nbsp;&nbsp;&nbsp;            a. **aws-marketplace:ViewSubscriptions**  
&nbsp;&nbsp;&nbsp;&nbsp;            b. **aws-marketplace:Unsubscribe**  
&nbsp;&nbsp;&nbsp;&nbsp;            c. **aws-marketplace:Subscribe**  
    B. or your AWS account has a subscription to For Seller to update: Quantum bin packing for vehicle routing.


### Contents

1. [Subscribe to the algorithm]()
2. [Prepare dataset]()  
    A. [Dataset format expected by the algorithm]()  
    B. [Configure dataset]()  
    C. [Upload datasets to Amazon S3]()  
3. [Execute optimization model]()  
    A. [Set up environment]()    
    B. [Execute model]()  
    C. [Visualize Output]()  
4. [Clean-up]()  
    A. [Unsubscribe to the listing (optional))]()

### 1. Subscribe to the algorithm

To subscribe to the algorithm:  
&nbsp; 1. Open the algorithm listing page **Quantum bin packing for vehicle routing**  
&nbsp; 2. On the AWS Marketplace listing, click on **Continue to subscribe** button.  
&nbsp; 3. On the **Subscribe to this software** page, review and click on "**Accept Offer**" if you agree with EULA, pricing, and support terms.  
&nbsp; 4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
algo_arn = "your algo arn"

### 2. Prepare dataset

In [ ]:
import os
import json 
import uuid
import boto3
import pickle
import base64
import tarfile
from pprint import pprint
from PIL import Image

import numpy as np
import pandas as pd

import urllib.request
from urllib.parse import urlparse

import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role

#### A. Dataset format expected by the algorithm

The algorithm requires data in the format as described for best results:  

&nbsp; 1. Input File name should be input_zip.zip.  
&nbsp; 2. The input data files must contain two csv files.  
&nbsp; 3. For detailed instructions, please refer sample notebook and algorithm input details.  

#### B. Configure dataset

In [15]:
training_dataset="Input/input_zip.zip"

#### C. Upload datasets to Amazon S3

In [16]:
role = get_execution_role()

In [ ]:
sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
bucket

In [ ]:
# training input location
common_prefix = "bin-packing"
training_input_prefix = common_prefix + "/training-input-data"
TRAINING_WORKDIR = "Input" #Input directory in Jupyter Server
training_input = sagemaker_session.upload_data(TRAINING_WORKDIR, key_prefix=training_input_prefix) #uploads data from jupyter server to S3
print("Training input uploaded to " + training_input)

### 3. Execute the training process

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to execute a training pipeline to get clean sentiment class labels using clean-sentiment-classification-labels Algorithm.

#### 3.1 Set up environment

In [19]:
output_location = 's3://{}/bin-packing/{}'.format(bucket, 'Output')

#### 3.2 Execute model

For information on creating an Estimator object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [20]:
training_instance_type="ml.m5.4xlarge"

In [21]:
#Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="bin-packing-v3",
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type=training_instance_type
)

#Run the training job.
estimator.fit({"training": training_input})

INFO:sagemaker:Creating training-job with name: bin-packing-v3-2025-03-05-12-32-34-584


2025-03-05 12:32:35 Starting - Starting the training job...
2025-03-05 12:33:07 Downloading - Downloading input data...
2025-03-05 12:33:27 Training - Training image download completed. Training in progress...Number of cases: 51
Minimum Number of bins required: 1
  Binary    Integer    Continuous    Quad    Linear    One-hot    EQ      LT    GT
--------  ---------  ------------  ------  --------  ---------  ------  ----  ----
    7956          0           154       0     10156       1326    2353  7803     0
Saved solution to /opt/ml/model/output_zip/Output.txt
Folder 'output_zip' has been zipped into '/opt/ml/model/output_zip.zip'.

2025-03-05 12:36:33 Uploading - Uploading generated training model
2025-03-05 12:36:46 Completed - Training job completed
Training seconds: 220
Billable seconds: 220


See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/jobs/) and monitor the metrics/logs in **Monitor** section.

In [ ]:
#output is available on following path
estimator.output_path

## Note: Inferencing is done within training pipeline. Real time inference endpoint/batch transform job is not required.

#### 3.3 Inspect the Output in S3

In [23]:
parsed_url = urlparse(estimator.output_path)
bucket_name = parsed_url.netloc
file_key = parsed_url.path[1:]+'/'+estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

s3_client = sagemaker_session.boto_session.client('s3')
response = s3_client.get_object(Bucket = sagemaker_session.default_bucket(), Key = file_key) 

In [24]:
bucketFolder = estimator.output_path.rsplit('/')[3] +'/Output/'+ estimator.latest_training_job.job_name+'/output/'+"model.tar.gz"

In [ ]:
s3_conn = boto3.client("s3")
bucket_name=bucket
with open('output.tar.gz', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder, f)
    print("Output file loaded from bucket")

In [28]:
with tarfile.open('output.tar.gz') as file:
    file.extractall('./output') 

In [ ]:
output_path = "output/"
file_path = os.path.join(output_path, "output.txt")

with open(file_path, "r") as file:
    content = file.read()

print(content)

### Clean-up

#### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://us-east-2.console.aws.amazon.com/sagemaker/home?region=us-east-2#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model.

#### Steps to unsubscribe to product from AWS Marketplace:

&nbsp; 1. Navigate to **Machine Learning tab** on **Your Software subscriptions page**.  
&nbsp; 2. Locate the listing that you want to cancel the subscription for, and then choose **Cancel Subscription** to cancel the subscription.